In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/openai/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet")

In [ ]:
df

,task_id,prompt,canonical_solution,test,entry_point
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation
...,...,...,...,...,...
159,HumanEval/159,"\ndef eat(number, need, remaining):\n """"""\n...",if(need <= remaining):\n return [ n...,def check(candidate):\n\n # Check some simp...,eat
160,HumanEval/160,"\ndef do_algebra(operator, operand):\n """"""\...",expression = str(operand[0])\n for oprt...,def check(candidate):\n\n # Check some simp...,do_algebra
161,HumanEval/161,"\ndef solve(s):\n """"""You are given a string...",flg = 0\n idx = 0\n new_str = list(s...,def check(candidate):\n\n # Check some simp...,solve
162,HumanEval/162,"\ndef string_to_md5(text):\n """"""\n Given...",import hashlib\n return hashlib.md5(tex...,def check(candidate):\n\n # Check some simp...,string_to_md5


In [ ]:
prompts = df["prompt"].tolist()
solutions = df["canonical_solution"].tolist()


In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = embedder.encode(prompts, batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:


import chromadb

client = chromadb.Client()
collection = client.create_collection(name="humaneval2")

for idx, (emb, prompt, solution) in enumerate(zip(embeddings, prompts, solutions)):
    collection.add(
        ids=[str(idx)],
        embeddings=[emb.tolist()],
        metadatas=[{"prompt": prompt, "solution": solution}])

In [ ]:
def retrieve_similar_context(query, k=1):
    query_emb = embedder.encode([query]).tolist()
    results = collection.query(query_embeddings=query_emb, n_results=k)
    return results["metadatas"]


In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import InferenceClient

load_dotenv()
API_KEY = os.getenv("HF_API_KEY")

client = InferenceClient(
    provider="nscale",
    api_key=API_KEY
)

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    messages=[
        {
            "role": "user",
            "content": "generate python code that return The sum of two numbers"
        }
    ],
)

print(completion.choices[0].message.content)



To solve this problem, we need to create a Python function that returns the sum of two numbers. 

### Approach
1. **Define the function**: We will define a function named `sum_two_numbers` that takes two parameters, `a` and `b`.
2. **Calculate the sum**: Inside the function, we will compute the sum of `a` and `b`.
3. **Return the result**: The computed sum will be stored in a variable and returned as the output of the function.
4. **Docstring**: Include a description of what the function does, its parameters, and what it returns.

### Solution Code

```python
def sum_two_numbers(a, b):
    total = a + b
    return total
```

### Explanation
- **Function Definition**: The function `sum_two_numbers` is defined to take two parameters, `a` and `b`.
- **Sum Calculation**: We compute the sum of `a` and `b` using the expression `a + b` and store the result in the variable `total`.
- **Return Statement**: The variable `total` is returned as the output of the function.

This implementation is

In [ ]:
def generate_code(user_prompt, k=1):
    retrieved = retrieve_similar_context(user_prompt, k)[0]
    context = "\n\n".join([f"# Task:\n{r['prompt']}\n{r['solution']}" for r in retrieved])

    final_prompt = f"{context}\n\n# Your Task:\n{user_prompt}\n"
    client = InferenceClient(
        provider="nscale",
        api_key=API_KEY
    )

    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        messages=[
            {
                "role": "user",
                "content": f"{final_prompt}"
            }
        ],
    )

    return completion.choices[0].message.content

ChatCompletionOutputMessage(role='assistant', content='\n\nTo generate a Python program that returns the sum of two numbers, follow these steps:\n\n### Step-by-Step Explanation\n\n1. **Define the Numbers**: Use variables to store the two numbers. These can be any numeric type.\n\n2. **Calculate the Sum**: Add the two numbers using the `+` operator.\n\n3. **Display the Result**: Use the `print()` function to show the result.\n\n### Code\n\n```python\n# Function to add two numbers and return the result\ndef add_two_numbers(num1, num2):\n    # Calculate the sum\n    sum_result = num1 + num2\n    # Display the result\n    print(f"The sum of {num1} and {num2} is {sum_result}")\n\n# Example usage\nprint("Adding 4 and 6.")\nadd_two_numbers(4, 6)  # Output: The sum of 4 and 6 is 10.\n```\n\n### Explanation\n\n- **Define Variables**: `num1` and `num2` are the two input numbers.\n- **Calculate Sum**: Add `num1` and `num2` to get `sum_result`.\n- **Print Result**: Use formatted string `f"{num1} a

In [16]:
query = "Write a function that returns the factorial of a number"
generated_code = generate_code(query)
print(generated_code)



To solve this problem, we need to compute the special factorial of a given integer \( n \). The special factorial is defined as the product of the factorials of all integers from 1 to \( n \). 

### Approach
The approach to compute the special factorial involves iterating through each integer from 1 to \( n \) and maintaining a running product of the current factorial values. Specifically, for each integer \( i \) in the range from 1 to \( n \), we compute the factorial of \( i \) and multiply it with the special factorial result. 

1. **Initialize Variables**: We start with two variables, `fact_i` and `special_fact`. `fact_i` will store the factorial of the current integer \( i \), and `special_fact` will store the product of all these factorials.
2. **Iterate and Update**: For each integer \( i \) from 1 to \( n \):
   - Multiply `fact_i` by the current \( i \) to get the factorial of \( i \) (which is updated in each iteration).
   - Multiply the updated `fact_i` with `special_fac

In [17]:
query = "Write a function that returns the factorial of a number"
generated_code = generate_code(query)
print(generated_code)



To solve this problem, we need to calculate the "Brazilian factorial" of a given integer \( n \). This type of factorial is defined as the product of the factorials of integers from 1 up to \( n \). 

### Approach
The task is to compute the product of factorials in a sequence starting from \( 1! \) up to \( n! \). This can be efficiently done using a loop that iterates from 1 to \( n \) and accumulates the product of the factorials found at each step.

1. **Initialization**: Start with `special_fact` as 1 to hold the cumulative product of the factorials.
2. **Iterate from 1 to n**: For each integer \( i \) from 1 to \( n \):
   - Compute the factorial of \( i \) (i.e., \( i! \)).
   - Multiply this factorial result by the current value of `special_fact` to accumulate the result.
3. **Return Result**: After completing the loop, return the accumulated product.

This approach ensures that we compute each factorial step-by-step and multiply them together, resulting in the final product.


In [1]:
  from pipeline import CodeGenPipeline
  pipeline = CodeGenPipeline("hf://datasets/openai/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet")
  result = pipeline.generate_code_from_prompt("Write a function that returns the factorial of a number")
  print(result)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]



To solve this problem, we need to compute the "Brazilian factorial" of a given integer n. The Brazilian factorial is defined as the product of the factorials of each integer from 1 to n. Specifically, it is the product of 1!, 2!, 3!, ..., n!.

### Approach
The approach to solve this problem involves the following steps:
1. **Initialization**: Start with two variables, `fact_i` and `special_fact`, both initialized to 1.
2. **Iterate through the range**: Loop through each integer i from 1 to n (inclusive).
3. **Update `fact_i`**: For each i, update `fact_i` by multiplying it with i. This effectively computes the factorial of i, i!.
4. **Update `special_fact`**: After updating `fact_i`, multiply the current value of `fact_i` with `special_fact` and store the result back in `special_fact`.
5. **Return the result**: After completing the loop through all integers from 1 to n, return the computed value of `special_fact`.

This approach ensures that each factorial from 1! up to n! is compute